In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install torch

In [ ]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub

In [3]:
#!pip install llama-cpp-python

In [4]:
from huggingface_hub import hf_hub_download

/opt/anaconda3/envs/viveksdmlenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_cpp import Llama

In [18]:
#download the model llama 2 7b
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in bin format

In [19]:

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/viveksd/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [20]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/viveksd/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.di

llama_model_loader: - kv  12:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   81 tensors
llama_model_loader: - type q5_K:  281 tensors
llama_model_loader: - type q6_K:    1 tensors
llm_load_vocab: special tokens definition check successful ( 259/32000 ).
llm_load_prin

In [54]:
prompt = "1000 + -2300 = ?"
prompt_template=f'''SYSTEM: You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 2+3=? you should answer 5. Let's start.

USER: {prompt}

ASSISTANT:
'''

In [57]:
response=lcpp_llm(prompt=prompt_template, max_tokens=100, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       8.56 ms /    23 runs   (    0.37 ms per token,  2686.29 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   15744.73 ms /    23 runs   (  684.55 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =   15798.74 ms /    24 tokens


In [58]:
print(response["choices"][0]["text"])


The sum of 1000 and -2300 is -1300.


In [60]:
import json

In [61]:
#load file in data folder int_addition.json
with open('int_addition.json') as f:
    data = json.load(f)

In [62]:
import time
#time sleep to avoid rate limit

for i in range(len(data)):
    if i%100==0:
      print(f'processing {i}th element')
    try:


        prompt = f"{data[i][0]} + {data[i][1]} ?"
        prompt_template=f'''SYSTEM: You are a math assistant. I will ask you some addition questions. For example, if I ask 'What is 2 + 3?', you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line.
        USER: {prompt}

        ASSISTANT:
        '''

        response=lcpp_llm(prompt=prompt_template, max_tokens=100, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
        #for each response extract the answer and write it to a file
        with open('int_addition.txt', 'a') as f:
            answer = response["choices"][0]["text"]
            
            f.write(f'{data[i]} = {answer}\n')
    except:
        print('rate limit reached. Sleeping for 2 minutes')
        time.sleep(2)

processing 0th element


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       3.06 ms /     8 runs   (    0.38 ms per token,  2613.52 tokens per second)
llama_print_timings: prompt eval time =    5405.65 ms /    66 tokens (   81.90 ms per token,    12.21 tokens per second)
llama_print_timings:        eval time =    4930.14 ms /     7 runs   (  704.31 ms per token,     1.42 tokens per second)
llama_print_timings:       total time =   10355.31 ms /    73 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       3.07 ms /     8 runs   (    0.38 ms per token,  2606.71 tokens per second)
llama_print_timings: prompt eval time =    1144.60 ms /    15 tokens (   76.31 ms per token,    13.11 tokens per second)
llama_print_timings:        eval time =    4929.48 ms /     7 runs   (  704.21 ms per token,     1.42 tokens per second)
llama_print_timings:       to

processing 100th element



llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       3.05 ms /     8 runs   (    0.38 ms per token,  2626.40 tokens per second)
llama_print_timings: prompt eval time =    1252.80 ms /    15 tokens (   83.52 ms per token,    11.97 tokens per second)
llama_print_timings:        eval time =    4781.97 ms /     7 runs   (  683.14 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =    6055.10 ms /    22 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       3.03 ms /     8 runs   (    0.38 ms per token,  2641.14 tokens per second)
llama_print_timings: prompt eval time =    1243.17 ms /    15 tokens (   82.88 ms per token,    12.07 tokens per second)
llama_print_timings:        eval time =    4846.75 ms /     7 runs   (  692.39 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =    6109.08 ms /    22 

processing 200th element


...

processing 2300th element



llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       8.79 ms /    25 runs   (    0.35 ms per token,  2844.79 tokens per second)
llama_print_timings: prompt eval time =    1283.11 ms /    25 tokens (   51.32 ms per token,    19.48 tokens per second)
llama_print_timings:        eval time =   16170.29 ms /    24 runs   (  673.76 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   17513.62 ms /    49 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       8.55 ms /    24 runs   (    0.36 ms per token,  2806.03 tokens per second)
llama_print_timings: prompt eval time =    1266.16 ms /    25 tokens (   50.65 ms per token,    19.74 tokens per second)
llama_print_timings:        eval time =   15446.44 ms /    23 runs   (  671.58 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   16769.95 ms /    48 

processing 2400th element



llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       9.05 ms /    26 runs   (    0.35 ms per token,  2872.93 tokens per second)
llama_print_timings: prompt eval time =    1386.06 ms /    28 tokens (   49.50 ms per token,    20.20 tokens per second)
llama_print_timings:        eval time =   16834.42 ms /    25 runs   (  673.38 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   18282.68 ms /    53 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4574.21 ms
llama_print_timings:      sample time =       9.09 ms /    26 runs   (    0.35 ms per token,  2860.60 tokens per second)
llama_print_timings: prompt eval time =    1348.51 ms /    27 tokens (   49.94 ms per token,    20.02 tokens per second)
llama_print_timings:        eval time =   16593.36 ms /    25 runs   (  663.73 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =   18005.61 ms /    52 

: 